**Proyecto:** INE Incidentes de Transito.

**Objetivo:** Transformaciones

**Dependencias / Notebooks:** ingest_fallecidos_lesionados, ingest_hechos_transito, inges_vehiculos

**Ambiente de Ejecución:** Databricks, Azure, Desarrollo

---
| Fecha | Desarrollador | Descripcion |
| :--- | :--- | :--- |
| 06/NOV/20025 | Ruben Coloma | Emision Inicial |

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
# Constantes
str_catalogo = "cat_ine_trafico_dev"
str_esquema = "bronze_ine_trafico_dev"

### Funciones UDF

In [0]:
def hora_missing_values(row_value):
    # Manejo de missing values para la hora de ocurrencia del incidente.
    if row_value == "Ignorada":
        return "-1"
    else:
        return row_value

In [0]:
def zona_missing_values(row_value):
    # Manejo de missing values para la zona de ocurrencia del incidente.
    if row_value == "Ignorada":
        return "-1"
    else:
        return row_value

In [0]:
def edad_missing_values(row_value):
    # Manejo de missing values para la edad de la persona.
    if row_value == "Ignorada":
        return "-1"
    else:
        return row_value

In [0]:
# Asignacion de funciones UDF
hora_missing_values_udf = F.udf(hora_missing_values, StringType())
zona_missing_values_udf = F.udf(zona_missing_values, StringType())
edad_missing_values_udf = F.udf(edad_missing_values, StringType())

### Fallecidos y Lesionados
**Tramiento a columnas numéricas con missing values:**
- hora_ocu
- zona_ocu
- edad_per

In [0]:
df_aux_fall_les = spark.table(f"{str_catalogo}.{str_esquema}.ine_aux_fallecitos_lesionados")

In [0]:
# Transformacion columna 'hora_ocu' con funsion UDF
df_aux_fall_les = df_aux_fall_les.withColumn("hora_ocu", hora_missing_values_udf("hora_ocu"))
# Transformacion columna 'zona_ocu' con funsion UDF
df_aux_fall_les = df_aux_fall_les.withColumn("zona_ocu", hora_missing_values_udf("zona_ocu"))
# Transformacion columna 'edad_ocu' con funsion UDF
df_aux_fall_les = df_aux_fall_les.withColumn("edad_per", hora_missing_values_udf("edad_per"))

In [0]:
# Convertir columnas a tipo ENTERO
df_aux_fall_les = df_aux_fall_les \
    .withColumn("hora_ocu", 
                regexp_replace(col("hora_ocu"), "[^0-9-]", "").cast("integer")) \
    .withColumn("zona_ocu", 
                regexp_replace(col("zona_ocu"), "[^0-9-]", "").cast("integer")) \
    .withColumn("edad_per", 
                regexp_replace(col("edad_per"), "[^0-9-]", "").cast("integer"))

In [0]:
# th_fallecidos_lesionados: Guardar los datos
df_aux_fall_les.write.mode("overwrite").saveAsTable("cat_ine_trafico_dev.silver_ine_trafico_dev.th_fallecidos_lesionados")

### Hechos de Transito
**Tramiento a columnas numéricas con missing values:**
- hora_ocu

In [0]:
# Obtener datos cargados
df_aux_hechos_tran = spark.table(f"{str_catalogo}.{str_esquema}.ine_aux_hechos_transito")

In [0]:
# Transformacion columna 'hora_ocu' con funsion UDF
df_aux_hechos_tran = df_aux_hechos_tran.withColumn("hora_ocu", hora_missing_values_udf("hora_ocu"))

In [0]:
# Transformar a tipo entero
df_aux_hechos_tran = df_aux_hechos_tran \
    .withColumn("hora_ocu", 
                regexp_replace(col("hora_ocu"), "[^0-9-]", "").cast("integer"))

In [0]:
# th_hechos_transito: Guardar los datos
df_aux_hechos_tran.write.mode("overwrite").saveAsTable("cat_ine_trafico_dev.silver_ine_trafico_dev.th_hechos_transito")

### Vehiculos Involucrados
**Tramiento a columnas numéricas con missing values:**
- hora_ocu

In [0]:
# Obtener datos cargados
df_aux_vehiculos = spark.table(f"{str_catalogo}.{str_esquema}.ine_aux_vehiculos")

In [0]:
# Transformacion columna 'hora_ocu' con funsion UDF
df_aux_vehiculos = df_aux_vehiculos.withColumn("hora_ocu", hora_missing_values_udf("hora_ocu"))

In [0]:
# Transformar a tipo entero
df_aux_vehiculos = df_aux_vehiculos \
    .withColumn("hora_ocu", 
                regexp_replace(col("hora_ocu"), "[^0-9-]", "").cast("integer"))

In [0]:
# th_vehiculos_involucrados: Guardar los datos
df_aux_vehiculos.write.mode("overwrite").saveAsTable("cat_ine_trafico_dev.silver_ine_trafico_dev.th_vehiculos_involucrados")